# 05. 자율 에이전트 (Autonomous Agents) 시스템

## 🎯 학습 목표
1. AI 에이전트의 개념과 구조 이해
2. ReAct 프레임워크 구현
3. 도구 사용과 메모리 시스템 구축
4. 다중 에이전트 협업 시스템 개발

## 📚 AI 에이전트란?

## 1. AI 에이전트 아키텍처

### 에이전트의 핵심 구성 요소

```
┌────────────────────────────────────────────────┐
│           Autonomous Agent System              │
├────────────────────────────────────────────────┤
│                                                │
│   ┌──────────┐      ┌──────────┐             │
│   │ Planning │  →   │ Acting   │              │
│   │  (계획)   │      │  (실행)   │              │
│   └──────────┘      └──────────┘             │
│         ↑                ↓                    │
│   ┌──────────┐      ┌──────────┐             │
│   │ Reasoning│  ←   │Observing │              │
│   │  (추론)   │      │  (관찰)   │              │
│   └──────────┘      └──────────┘             │
│                                                │
│   Supporting Components:                       │
│   • Memory (기억)                              │
│   • Tools (도구)                               │
│   • Knowledge (지식)                           │
└────────────────────────────────────────────────┘
```

### 에이전트 vs 일반 LLM

| 특징 | 일반 LLM | AI 에이전트 |
|------|----------|------------|
| 자율성 | ❌ 수동적 응답 | ✅ 능동적 행동 |
| 도구 사용 | ❌ 텍스트만 | ✅ 다양한 도구 활용 |
| 메모리 | ❌ 컨텍스트 제한 | ✅ 장기 메모리 |
| 계획 수립 | ❌ 단일 응답 | ✅ 다단계 계획 |
| 자기 반성 | ❌ | ✅ 결과 평가 및 개선 |

### 에이전트 레벨

1. **Level 1 - Tool Caller**: 도구 호출 가능
2. **Level 2 - Reasoner**: 추론 후 행동
3. **Level 3 - Planner**: 계획 수립 및 실행
4. **Level 4 - Self-Improver**: 자기 개선 가능

## 2. ReAct 프레임워크

### ReAct (Reasoning + Acting)란?

**핵심**: 생각(Thought) → 행동(Action) → 관찰(Observation) 사이클

```
사용자 질문: "오늘 서울 날씨는 어때?"

Thought 1: 현재 날씨 정보를 얻어야 한다
Action 1: weather_tool
Action Input: {"city": "Seoul"}
Observation 1: 맑음, 15°C, 습도 45%

Thought 2: 날씨 정보를 사용자 친화적으로 설명해야 한다
Final Answer: 오늘 서울은 맑고 따뜻한 날씨입니다. 
             기온은 15도로 쾌적하며, 습도도 45%로 적당합니다.
```

## 3. 환경 설정

In [ ]:
import json
import re
from typing import List, Dict, Any, Optional, Callable
from dataclasses import dataclass
from enum import Enum
from datetime import datetime
import requests
import numpy as np

print("✅ 패키지 로드 완료")

## 4. 기본 도구(Tools) 구현

In [ ]:
class ToolKit:
    """에이전트가 사용할 수 있는 도구 모음"""
    
    @staticmethod
    def calculator(expression: str) -> float:
        """수학 계산 도구"""
        try:
            # 안전한 계산을 위해 eval 제한
            allowed_names = {
                "abs": abs, "round": round, "min": min, "max": max,
                "sum": sum, "pow": pow, "sqrt": np.sqrt
            }
            result = eval(expression, {"__builtins__": {}}, allowed_names)
            return f"계산 결과: {result}"
        except Exception as e:
            return f"계산 오류: {str(e)}"
    
    @staticmethod
    def get_datetime(format: str = "%Y-%m-%d %H:%M:%S") -> str:
        """현재 날짜와 시간 반환"""
        return datetime.now().strftime(format)
    
    @staticmethod
    def search_web(query: str) -> str:
        """웹 검색 시뮬레이션"""
        # 실제로는 검색 API 사용
        mock_results = {
            "python": "Python은 고급 프로그래밍 언어로, 간결한 문법과 강력한 기능을 제공합니다.",
            "machine learning": "머신러닝은 데이터에서 패턴을 학습하는 AI 기술입니다.",
            "날씨": "오늘은 맑고 기온은 20도입니다.",
            "주식": "KOSPI 2,500포인트, NASDAQ 15,000포인트"
        }
        
        for key, value in mock_results.items():
            if key in query.lower():
                return value
        return "검색 결과를 찾을 수 없습니다."
    
    @staticmethod
    def read_file(filepath: str) -> str:
        """파일 읽기"""
        try:
            with open(filepath, 'r', encoding='utf-8') as f:
                return f.read()[:500]  # 처음 500자만
        except Exception as e:
            return f"파일 읽기 오류: {str(e)}"
    
    @staticmethod
    def write_file(filepath: str, content: str) -> str:
        """파일 쓰기"""
        try:
            with open(filepath, 'w', encoding='utf-8') as f:
                f.write(content)
            return f"파일 저장 완료: {filepath}"
        except Exception as e:
            return f"파일 쓰기 오류: {str(e)}"
    
    @staticmethod
    def send_email(to: str, subject: str, body: str) -> str:
        """이메일 전송 시뮬레이션"""
        return f"이메일 전송 완료: {to}에게 '{subject}' 제목으로 전송"

# 도구 테스트
toolkit = ToolKit()

print("🔧 도구 테스트:")
print(f"계산: {toolkit.calculator('2 + 3 * 4')}")
print(f"시간: {toolkit.get_datetime()}")
print(f"검색: {toolkit.search_web('python')}")

## 5. ReAct 에이전트 구현

In [ ]:
class ReActAgent:
    """ReAct 패턴 기반 에이전트"""
    
    def __init__(self, tools: Dict[str, Callable] = None):
        self.tools = tools or {}
        self.history = []
        self.max_iterations = 5
        
        # 기본 도구 등록
        self._register_default_tools()
    
    def _register_default_tools(self):
        """기본 도구 등록"""
        toolkit = ToolKit()
        self.tools = {
            "calculator": toolkit.calculator,
            "datetime": toolkit.get_datetime,
            "search": toolkit.search_web,
            "read_file": toolkit.read_file,
            "write_file": toolkit.write_file
        }
    
    def think(self, question: str, context: str = "") -> str:
        """추론 단계"""
        # 실제로는 LLM을 사용하여 추론
        # 여기서는 규칙 기반 시뮬레이션
        
        if "계산" in question or any(op in question for op in ["+", "-", "*", "/"]):
            return "수학 계산이 필요합니다."
        elif "날짜" in question or "시간" in question or "오늘" in question:
            return "현재 시간 정보가 필요합니다."
        elif "검색" in question or "찾아" in question:
            return "웹 검색이 필요합니다."
        elif context:
            return "컨텍스트를 바탕으로 답변을 생성합니다."
        else:
            return "직접 답변이 가능합니다."
    
    def act(self, thought: str, question: str) -> tuple:
        """행동 단계"""
        # 생각에 따라 적절한 도구 선택
        
        if "계산" in thought:
            # 질문에서 수식 추출
            import re
            numbers = re.findall(r'[\d+\-*/\s()]+', question)
            if numbers:
                return "calculator", {"expression": numbers[0]}
        
        elif "시간" in thought:
            return "datetime", {"format": "%Y-%m-%d %H:%M:%S"}
        
        elif "검색" in thought:
            # 검색어 추출
            keywords = question.split()[-2:]  # 마지막 2단어
            return "search", {"query": " ".join(keywords)}
        
        return None, None
    
    def observe(self, tool_name: str, tool_input: Dict) -> str:
        """관찰 단계 (도구 실행)"""
        if tool_name in self.tools:
            tool = self.tools[tool_name]
            try:
                result = tool(**tool_input)
                return str(result)
            except Exception as e:
                return f"도구 실행 오류: {str(e)}"
        return "도구를 찾을 수 없습니다."
    
    def run(self, question: str) -> str:
        """에이전트 실행"""
        print(f"\n🤖 ReAct Agent 처리: {question}")
        print("="*50)
        
        context = ""
        
        for i in range(self.max_iterations):
            print(f"\n[Iteration {i+1}]")
            
            # 1. Think
            thought = self.think(question, context)
            print(f"💭 Thought: {thought}")
            self.history.append({"type": "thought", "content": thought})
            
            # 2. Act
            tool_name, tool_input = self.act(thought, question)
            
            if tool_name:
                print(f"🔧 Action: {tool_name}")
                print(f"   Input: {tool_input}")
                self.history.append({
                    "type": "action",
                    "tool": tool_name,
                    "input": tool_input
                })
                
                # 3. Observe
                observation = self.observe(tool_name, tool_input)
                print(f"👀 Observation: {observation}")
                self.history.append({
                    "type": "observation",
                    "content": observation
                })
                
                context += f"\n{observation}"
            else:
                # 최종 답변 생성
                if context:
                    answer = f"결과: {context}"
                else:
                    answer = "답변을 생성할 수 없습니다."
                
                print(f"\n✅ Final Answer: {answer}")
                return answer
        
        return f"최대 반복 횟수 도달. 현재 컨텍스트: {context}"

# 에이전트 테스트
agent = ReActAgent()

# 테스트 질문들
questions = [
    "25 * 4 + 10은 얼마야?",
    "오늘 날짜가 뭐야?",
    "Python에 대해 검색해줘"
]

for q in questions:
    result = agent.run(q)
    print("\n" + "="*50)

## 6. 메모리 시스템 구현

In [ ]:
class MemorySystem:
    """에이전트 메모리 시스템"""
    
    def __init__(self):
        # 다양한 메모리 타입
        self.short_term = []  # 단기 메모리 (현재 대화)
        self.long_term = {}   # 장기 메모리 (영구 저장)
        self.working = {}     # 작업 메모리 (임시 저장)
        self.episodic = []    # 에피소드 메모리 (경험)
        
        # 메모리 제한
        self.short_term_limit = 10
        self.episodic_limit = 100
    
    def add_to_short_term(self, item: Dict):
        """단기 메모리에 추가"""
        self.short_term.append({
            **item,
            "timestamp": datetime.now().isoformat()
        })
        
        # 메모리 제한 관리
        if len(self.short_term) > self.short_term_limit:
            # 오래된 항목을 장기 메모리로 이동
            old_item = self.short_term.pop(0)
            self._move_to_long_term(old_item)
    
    def _move_to_long_term(self, item: Dict):
        """장기 메모리로 이동"""
        key = f"memory_{len(self.long_term)}"
        self.long_term[key] = item
    
    def remember(self, key: str, value: Any):
        """장기 메모리에 저장"""
        self.long_term[key] = {
            "value": value,
            "created_at": datetime.now().isoformat(),
            "access_count": 0
        }
    
    def recall(self, key: str) -> Any:
        """장기 메모리에서 회상"""
        if key in self.long_term:
            memory = self.long_term[key]
            
            # 접근 횟수 증가
            if isinstance(memory, dict) and "access_count" in memory:
                memory["access_count"] += 1
                return memory["value"]
            
            return memory
        return None
    
    def add_episode(self, episode: Dict):
        """에피소드 메모리에 추가"""
        self.episodic.append({
            **episode,
            "timestamp": datetime.now().isoformat()
        })
        
        # 제한 관리
        if len(self.episodic) > self.episodic_limit:
            self.episodic.pop(0)
    
    def search_memory(self, query: str, memory_type: str = "all") -> List[Dict]:
        """메모리 검색"""
        results = []
        
        if memory_type in ["all", "short_term"]:
            for item in self.short_term:
                if query.lower() in str(item).lower():
                    results.append({"type": "short_term", "content": item})
        
        if memory_type in ["all", "long_term"]:
            for key, value in self.long_term.items():
                if query.lower() in key.lower() or query.lower() in str(value).lower():
                    results.append({"type": "long_term", "key": key, "content": value})
        
        if memory_type in ["all", "episodic"]:
            for episode in self.episodic:
                if query.lower() in str(episode).lower():
                    results.append({"type": "episodic", "content": episode})
        
        return results
    
    def get_context(self, n_recent: int = 5) -> str:
        """최근 컨텍스트 생성"""
        recent_items = self.short_term[-n_recent:]
        context_parts = []
        
        for item in recent_items:
            if isinstance(item, dict):
                content = item.get("content", item.get("value", str(item)))
            else:
                content = str(item)
            context_parts.append(content)
        
        return "\n".join(context_parts)
    
    def summarize(self) -> Dict:
        """메모리 상태 요약"""
        return {
            "short_term_count": len(self.short_term),
            "long_term_count": len(self.long_term),
            "working_keys": list(self.working.keys()),
            "episodic_count": len(self.episodic),
            "total_memories": len(self.short_term) + len(self.long_term) + len(self.episodic)
        }

# 메모리 시스템 테스트
memory = MemorySystem()

# 다양한 정보 저장
memory.remember("user_name", "Alice")
memory.remember("favorite_color", "blue")
memory.add_to_short_term({"role": "user", "content": "안녕하세요"})
memory.add_to_short_term({"role": "assistant", "content": "안녕하세요! 무엇을 도와드릴까요?"})
memory.add_episode({"task": "greeting", "success": True})

print("🧠 메모리 시스템 상태:")
print(json.dumps(memory.summarize(), indent=2, ensure_ascii=False))

print("\n📝 사용자 이름 회상:", memory.recall("user_name"))
print("\n🔍 '안녕' 검색 결과:")
for result in memory.search_memory("안녕"):
    print(f"  - [{result['type']}] {result.get('content', result.get('key'))}")

## 7. 고급 에이전트: 계획 수립과 실행

In [ ]:
class PlanningAgent:
    """계획을 수립하고 실행하는 에이전트"""
    
    def __init__(self):
        self.memory = MemorySystem()
        self.tools = self._setup_tools()
        self.current_plan = []
        self.execution_history = []
    
    def _setup_tools(self):
        """도구 설정"""
        toolkit = ToolKit()
        return {
            "calculate": toolkit.calculator,
            "search": toolkit.search_web,
            "datetime": toolkit.get_datetime,
            "write_file": toolkit.write_file,
            "send_email": toolkit.send_email
        }
    
    def decompose_task(self, task: str) -> List[Dict]:
        """복잡한 작업을 작은 단계로 분해"""
        
        # 규칙 기반 분해 (실제로는 LLM 사용)
        steps = []
        
        if "보고서" in task:
            steps = [
                {"step": "정보 수집", "tool": "search", "status": "pending"},
                {"step": "데이터 분석", "tool": "calculate", "status": "pending"},
                {"step": "보고서 작성", "tool": "write_file", "status": "pending"},
                {"step": "이메일 전송", "tool": "send_email", "status": "pending"}
            ]
        elif "일정" in task:
            steps = [
                {"step": "현재 시간 확인", "tool": "datetime", "status": "pending"},
                {"step": "일정 검색", "tool": "search", "status": "pending"},
                {"step": "일정 정리", "tool": "write_file", "status": "pending"}
            ]
        elif "분석" in task:
            steps = [
                {"step": "데이터 수집", "tool": "search", "status": "pending"},
                {"step": "계산 수행", "tool": "calculate", "status": "pending"},
                {"step": "결과 저장", "tool": "write_file", "status": "pending"}
            ]
        else:
            steps = [
                {"step": "작업 분석", "tool": None, "status": "pending"},
                {"step": "실행", "tool": None, "status": "pending"}
            ]
        
        return steps
    
    def create_plan(self, goal: str) -> List[Dict]:
        """목표 달성을 위한 계획 생성"""
        
        print(f"\n📋 목표: {goal}")
        print("계획 수립 중...\n")
        
        # 작업 분해
        steps = self.decompose_task(goal)
        
        # 계획에 우선순위와 의존성 추가
        for i, step in enumerate(steps):
            step["id"] = i + 1
            step["priority"] = len(steps) - i  # 역순 우선순위
            step["dependencies"] = [i] if i > 0 else []  # 이전 단계 의존
        
        self.current_plan = steps
        
        # 계획 출력
        print("📝 생성된 계획:")
        for step in steps:
            deps = f" (의존: {step['dependencies']})" if step['dependencies'] else ""
            print(f"  {step['id']}. {step['step']} [{step['tool']}]{deps}")
        
        return steps
    
    def execute_step(self, step: Dict) -> Dict:
        """단일 단계 실행"""
        
        print(f"\n▶️ 실행: {step['step']}")
        
        result = {
            "step_id": step["id"],
            "step_name": step["step"],
            "status": "failed",
            "output": None,
            "error": None
        }
        
        try:
            if step["tool"] and step["tool"] in self.tools:
                # 도구 실행
                tool = self.tools[step["tool"]]
                
                # 간단한 입력 생성 (실제로는 컨텍스트 기반)
                if step["tool"] == "calculate":
                    output = tool("100 * 2.5")
                elif step["tool"] == "search":
                    output = tool("python machine learning")
                elif step["tool"] == "write_file":
                    output = tool("report.txt", "분석 결과: 완료")
                elif step["tool"] == "send_email":
                    output = tool("user@example.com", "보고서", "첨부파일 참조")
                else:
                    output = tool()
                
                result["output"] = output
                result["status"] = "completed"
                print(f"   ✅ 성공: {output}")
                
            else:
                # 도구 없이 실행
                result["output"] = f"{step['step']} 완료 (시뮬레이션)"
                result["status"] = "completed"
                print(f"   ✅ 완료")
                
        except Exception as e:
            result["error"] = str(e)
            print(f"   ❌ 오류: {e}")
        
        # 실행 이력 저장
        self.execution_history.append(result)
        
        # 메모리에 저장
        self.memory.add_episode(result)
        
        return result
    
    def execute_plan(self) -> Dict:
        """전체 계획 실행"""
        
        if not self.current_plan:
            return {"status": "error", "message": "계획이 없습니다"}
        
        print("\n🚀 계획 실행 시작")
        print("="*50)
        
        results = []
        
        for step in self.current_plan:
            # 의존성 확인
            if step["dependencies"]:
                deps_satisfied = all(
                    self.current_plan[dep]["status"] == "completed" 
                    for dep in step["dependencies"]
                )
                if not deps_satisfied:
                    print(f"\n⏸️ {step['step']}: 의존성 미충족")
                    step["status"] = "skipped"
                    continue
            
            # 단계 실행
            result = self.execute_step(step)
            results.append(result)
            
            # 상태 업데이트
            step["status"] = result["status"]
        
        # 최종 요약
        summary = {
            "total_steps": len(self.current_plan),
            "completed": sum(1 for r in results if r["status"] == "completed"),
            "failed": sum(1 for r in results if r["status"] == "failed"),
            "results": results
        }
        
        print("\n" + "="*50)
        print("📊 실행 요약:")
        print(f"  • 전체: {summary['total_steps']}개 단계")
        print(f"  • 완료: {summary['completed']}개")
        print(f"  • 실패: {summary['failed']}개")
        
        return summary

# 계획 에이전트 테스트
planner = PlanningAgent()

# 복잡한 작업 요청
goal = "주식 시장 분석 보고서를 작성하고 이메일로 전송해줘"

# 계획 수립
plan = planner.create_plan(goal)

# 계획 실행
execution_result = planner.execute_plan()

## 8. 다중 에이전트 시스템

In [ ]:
class MultiAgentSystem:
    """여러 에이전트가 협업하는 시스템"""
    
    def __init__(self):
        self.agents = {}
        self.communication_channel = []  # 에이전트 간 통신
        self.shared_memory = MemorySystem()  # 공유 메모리
        
        # 기본 에이전트 생성
        self._create_default_agents()
    
    def _create_default_agents(self):
        """기본 에이전트 생성"""
        
        # 1. 연구원 에이전트
        self.agents["researcher"] = {
            "role": "정보 수집 및 분석",
            "skills": ["search", "summarize", "analyze"],
            "status": "idle"
        }
        
        # 2. 개발자 에이전트
        self.agents["developer"] = {
            "role": "코드 작성 및 디버깅",
            "skills": ["code", "debug", "test"],
            "status": "idle"
        }
        
        # 3. 관리자 에이전트
        self.agents["manager"] = {
            "role": "작업 조율 및 품질 관리",
            "skills": ["plan", "coordinate", "review"],
            "status": "idle"
        }
        
        # 4. 검토자 에이전트
        self.agents["reviewer"] = {
            "role": "결과 검증 및 피드백",
            "skills": ["validate", "feedback", "improve"],
            "status": "idle"
        }
    
    def assign_task(self, task: str) -> str:
        """작업을 적절한 에이전트에게 할당"""
        
        # 작업 분석 (실제로는 더 복잡한 매칭)
        if "검색" in task or "조사" in task or "분석" in task:
            return "researcher"
        elif "코드" in task or "개발" in task or "프로그램" in task:
            return "developer"
        elif "계획" in task or "관리" in task:
            return "manager"
        elif "검토" in task or "피드백" in task:
            return "reviewer"
        else:
            return "manager"  # 기본값
    
    def communicate(self, from_agent: str, to_agent: str, message: str):
        """에이전트 간 통신"""
        
        communication = {
            "from": from_agent,
            "to": to_agent,
            "message": message,
            "timestamp": datetime.now().isoformat()
        }
        
        self.communication_channel.append(communication)
        print(f"📨 [{from_agent} → {to_agent}]: {message}")
    
    def collaborate(self, project: str) -> Dict:
        """프로젝트 협업 실행"""
        
        print(f"\n🤝 다중 에이전트 협업 프로젝트: {project}")
        print("="*60)
        
        # 1. 매니저가 계획 수립
        print("\n[Phase 1: 계획]")
        self.agents["manager"]["status"] = "working"
        plan = f"{project}를 위한 3단계 계획"
        self.communicate("manager", "all", f"계획 수립 완료: {plan}")
        self.shared_memory.remember("project_plan", plan)
        
        # 2. 연구원이 정보 수집
        print("\n[Phase 2: 연구]")
        self.agents["researcher"]["status"] = "working"
        research = f"{project}에 대한 연구 자료 수집 완료"
        self.communicate("researcher", "developer", research)
        self.shared_memory.remember("research_data", research)
        
        # 3. 개발자가 구현
        print("\n[Phase 3: 개발]")
        self.agents["developer"]["status"] = "working"
        implementation = f"{project} 구현 완료"
        self.communicate("developer", "reviewer", implementation)
        self.shared_memory.remember("implementation", implementation)
        
        # 4. 검토자가 검증
        print("\n[Phase 4: 검토]")
        self.agents["reviewer"]["status"] = "working"
        review = f"{project} 검토 완료 - 승인"
        self.communicate("reviewer", "manager", review)
        
        # 5. 매니저가 최종 보고
        print("\n[Phase 5: 완료]")
        self.communicate("manager", "all", f"프로젝트 {project} 성공적으로 완료!")
        
        # 모든 에이전트 상태 리셋
        for agent in self.agents.values():
            agent["status"] = "idle"
        
        # 결과 요약
        result = {
            "project": project,
            "status": "completed",
            "phases_completed": 5,
            "agents_involved": list(self.agents.keys()),
            "communications": len(self.communication_channel),
            "memories_created": self.shared_memory.summarize()["long_term_count"]
        }
        
        print("\n" + "="*60)
        print("📊 협업 결과:")
        print(json.dumps(result, indent=2, ensure_ascii=False))
        
        return result
    
    def get_status(self) -> Dict:
        """시스템 상태 확인"""
        return {
            "agents": [
                {
                    "name": name,
                    "role": agent["role"],
                    "status": agent["status"]
                }
                for name, agent in self.agents.items()
            ],
            "total_communications": len(self.communication_channel),
            "shared_memories": self.shared_memory.summarize()
        }

# 다중 에이전트 시스템 테스트
mas = MultiAgentSystem()

# 시스템 상태 확인
print("🏢 다중 에이전트 시스템")
print("="*60)
print("\n에이전트 목록:")
for name, agent in mas.agents.items():
    print(f"  • {name}: {agent['role']}")

# 프로젝트 실행
project_result = mas.collaborate("AI 챗봇 개발")

# 통신 기록 확인
print("\n💬 통신 기록:")
for comm in mas.communication_channel[-3:]:
    print(f"  [{comm['from']} → {comm['to']}]: {comm['message']}")

## 9. 자기 개선 에이전트

In [ ]:
class SelfImprovingAgent:
    """자기 개선 능력이 있는 에이전트"""
    
    def __init__(self):
        self.performance_history = []
        self.learned_patterns = {}
        self.improvement_strategies = []
        self.current_performance = 0.5  # 초기 성능
    
    def execute_task(self, task: str) -> Dict:
        """작업 실행"""
        
        # 시뮬레이션된 작업 실행
        import random
        
        # 학습된 패턴 적용
        performance_boost = 0
        if task in self.learned_patterns:
            performance_boost = self.learned_patterns[task]
        
        # 성공 확률 계산
        success_rate = min(1.0, self.current_performance + performance_boost)
        success = random.random() < success_rate
        
        result = {
            "task": task,
            "success": success,
            "performance_score": success_rate,
            "timestamp": datetime.now().isoformat()
        }
        
        # 결과 기록
        self.performance_history.append(result)
        
        return result
    
    def analyze_performance(self) -> Dict:
        """성능 분석"""
        
        if not self.performance_history:
            return {"message": "성능 데이터 없음"}
        
        # 성공률 계산
        total_tasks = len(self.performance_history)
        successful_tasks = sum(1 for r in self.performance_history if r["success"])
        success_rate = successful_tasks / total_tasks
        
        # 작업별 성능
        task_performance = {}
        for record in self.performance_history:
            task = record["task"]
            if task not in task_performance:
                task_performance[task] = {"attempts": 0, "successes": 0}
            
            task_performance[task]["attempts"] += 1
            if record["success"]:
                task_performance[task]["successes"] += 1
        
        # 개선이 필요한 영역 식별
        weak_areas = []
        for task, perf in task_performance.items():
            rate = perf["successes"] / perf["attempts"]
            if rate < 0.7:  # 70% 미만 성공률
                weak_areas.append({"task": task, "success_rate": rate})
        
        analysis = {
            "overall_success_rate": success_rate,
            "total_tasks": total_tasks,
            "task_performance": task_performance,
            "weak_areas": weak_areas
        }
        
        return analysis
    
    def generate_improvement_strategy(self, analysis: Dict) -> List[str]:
        """개선 전략 생성"""
        
        strategies = []
        
        # 전체 성공률 기반 전략
        if analysis["overall_success_rate"] < 0.5:
            strategies.append("기본 능력 강화 필요")
        elif analysis["overall_success_rate"] < 0.8:
            strategies.append("특정 영역 집중 개선")
        else:
            strategies.append("현재 성능 유지 및 최적화")
        
        # 약한 영역 개선 전략
        for weak in analysis.get("weak_areas", []):
            strategies.append(f"{weak['task']} 작업 패턴 학습 강화")
        
        self.improvement_strategies = strategies
        return strategies
    
    def self_improve(self) -> Dict:
        """자기 개선 실행"""
        
        print("\n🔄 자기 개선 프로세스")
        print("="*50)
        
        # 1. 성능 분석
        print("\n[Step 1: 성능 분석]")
        analysis = self.analyze_performance()
        print(f"  • 전체 성공률: {analysis.get('overall_success_rate', 0):.1%}")
        print(f"  • 총 작업 수: {analysis.get('total_tasks', 0)}")
        
        # 2. 개선 전략 생성
        print("\n[Step 2: 개선 전략]")
        strategies = self.generate_improvement_strategy(analysis)
        for strategy in strategies:
            print(f"  • {strategy}")
        
        # 3. 학습 패턴 업데이트
        print("\n[Step 3: 패턴 학습]")
        for task, perf in analysis.get("task_performance", {}).items():
            success_rate = perf["successes"] / perf["attempts"]
            
            # 성공률이 높은 패턴 강화
            if success_rate > 0.7:
                self.learned_patterns[task] = 0.1  # 보너스 추가
                print(f"  ✅ {task}: 패턴 강화 (+0.1)")
            elif success_rate < 0.3:
                self.learned_patterns[task] = 0.2  # 더 많은 보너스
                print(f"  📈 {task}: 집중 학습 (+0.2)")
        
        # 4. 전체 성능 업데이트
        print("\n[Step 4: 성능 업데이트]")
        old_performance = self.current_performance
        
        # 성공률에 따라 기본 성능 조정
        if analysis.get('overall_success_rate', 0) > 0.8:
            self.current_performance = min(1.0, self.current_performance + 0.05)
        elif analysis.get('overall_success_rate', 0) < 0.4:
            self.current_performance = max(0.3, self.current_performance - 0.05)
        
        print(f"  • 이전 성능: {old_performance:.2f}")
        print(f"  • 현재 성능: {self.current_performance:.2f}")
        print(f"  • 변화량: {self.current_performance - old_performance:+.2f}")
        
        return {
            "improved": self.current_performance > old_performance,
            "performance_change": self.current_performance - old_performance,
            "learned_patterns": len(self.learned_patterns),
            "strategies_applied": len(strategies)
        }

# 자기 개선 에이전트 테스트
sia = SelfImprovingAgent()

print("🤖 자기 개선 에이전트 시뮬레이션")
print("="*60)

# 여러 작업 실행
tasks = [
    "데이터 분석", "보고서 작성", "코드 리뷰",
    "데이터 분석", "버그 수정", "보고서 작성",
    "테스트 실행", "데이터 분석", "문서화"
]

print("\n작업 실행 중...")
for task in tasks:
    result = sia.execute_task(task)
    status = "✅" if result["success"] else "❌"
    print(f"  {status} {task}: 성능={result['performance_score']:.2f}")

# 자기 개선 실행
improvement_result = sia.self_improve()

print("\n" + "="*50)
print("🎯 개선 결과:")
print(json.dumps(improvement_result, indent=2, ensure_ascii=False))

## 10. 실전 프로젝트: 연구 어시스턴트 에이전트

In [ ]:
class ResearchAssistant:
    """연구를 도와주는 종합 어시스턴트 에이전트"""
    
    def __init__(self):
        self.memory = MemorySystem()
        self.tools = self._setup_tools()
        self.research_papers = []
        self.current_topic = None
    
    def _setup_tools(self):
        return {
            "search": self._search_papers,
            "summarize": self._summarize_text,
            "analyze": self._analyze_data,
            "visualize": self._create_visualization,
            "cite": self._generate_citation
        }
    
    def _search_papers(self, query: str) -> List[Dict]:
        """논문 검색 시뮬레이션"""
        papers = [
            {
                "title": f"Deep Learning in {query}",
                "authors": ["Smith, J.", "Lee, K."],
                "year": 2024,
                "abstract": f"A comprehensive study on {query} using deep learning..."
            },
            {
                "title": f"Advances in {query}: A Survey",
                "authors": ["Johnson, M."],
                "year": 2023,
                "abstract": f"This paper surveys recent advances in {query}..."
            }
        ]
        return papers
    
    def _summarize_text(self, text: str, max_length: int = 100) -> str:
        """텍스트 요약"""
        words = text.split()
        if len(words) <= max_length:
            return text
        return " ".join(words[:max_length]) + "..."
    
    def _analyze_data(self, data: List) -> Dict:
        """데이터 분석"""
        return {
            "count": len(data),
            "types": list(set(type(item).__name__ for item in data)),
            "summary": "Data analysis complete"
        }
    
    def _create_visualization(self, data_type: str) -> str:
        """시각화 생성 시뮬레이션"""
        return f"[{data_type} 차트 생성됨: chart_{datetime.now().strftime('%Y%m%d_%H%M%S')}.png]"
    
    def _generate_citation(self, paper: Dict) -> str:
        """인용 생성"""
        authors = ", ".join(paper["authors"])
        return f"{authors} ({paper['year']}). {paper['title']}."
    
    def conduct_research(self, topic: str) -> Dict:
        """연구 수행"""
        
        print(f"\n📚 연구 주제: {topic}")
        print("="*60)
        
        self.current_topic = topic
        research_results = {
            "topic": topic,
            "phases": [],
            "findings": [],
            "recommendations": []
        }
        
        # Phase 1: 문헌 조사
        print("\n[Phase 1: 문헌 조사]")
        papers = self._search_papers(topic)
        self.research_papers.extend(papers)
        print(f"  • {len(papers)}개 관련 논문 발견")
        
        for paper in papers[:2]:
            print(f"    - {paper['title']} ({paper['year']})")
            self.memory.remember(f"paper_{paper['year']}", paper)
        
        research_results["phases"].append({
            "name": "Literature Review",
            "papers_found": len(papers)
        })
        
        # Phase 2: 데이터 수집 및 분석
        print("\n[Phase 2: 데이터 분석]")
        analysis = self._analyze_data(papers)
        print(f"  • 데이터 포인트: {analysis['count']}")
        print(f"  • 분석 완료: {analysis['summary']}")
        
        research_results["phases"].append({
            "name": "Data Analysis",
            "analysis": analysis
        })
        
        # Phase 3: 핵심 발견사항 도출
        print("\n[Phase 3: 핵심 발견사항]")
        findings = [
            f"{topic}는 최근 급격한 발전을 보이고 있음",
            f"딥러닝 기술이 {topic} 분야에 혁신을 가져옴",
            "추가 연구가 필요한 영역 식별"
        ]
        
        for i, finding in enumerate(findings, 1):
            print(f"  {i}. {finding}")
            research_results["findings"].append(finding)
        
        # Phase 4: 시각화
        print("\n[Phase 4: 시각화]")
        viz1 = self._create_visualization("trend")
        viz2 = self._create_visualization("distribution")
        print(f"  • {viz1}")
        print(f"  • {viz2}")
        
        # Phase 5: 권장사항
        print("\n[Phase 5: 권장사항]")
        recommendations = [
            "최신 연구 동향 지속적 모니터링",
            "실험적 검증 수행 필요",
            "산업 파트너십 고려"
        ]
        
        for rec in recommendations:
            print(f"  • {rec}")
            research_results["recommendations"].append(rec)
        
        # 참고문헌 생성
        print("\n[참고문헌]")
        for paper in papers[:3]:
            citation = self._generate_citation(paper)
            print(f"  • {citation}")
        
        return research_results
    
    def generate_report(self) -> str:
        """연구 보고서 생성"""
        
        if not self.current_topic:
            return "연구 주제가 설정되지 않았습니다."
        
        report = f"""
# 연구 보고서: {self.current_topic}

## 요약
{self.current_topic}에 대한 종합적인 연구를 수행했습니다.

## 방법론
- 문헌 조사
- 데이터 분석
- 전문가 검토

## 주요 발견사항
{chr(10).join(f'- {f}' for f in self.memory.search_memory(self.current_topic)[:3])}

## 결론
본 연구는 {self.current_topic} 분야의 중요한 인사이트를 제공합니다.

## 향후 연구 방향
- 실증적 검증
- 확장 연구
- 응용 개발
"""
        
        return report

# 연구 어시스턴트 테스트
assistant = ResearchAssistant()

# 연구 수행
research_results = assistant.conduct_research("자연어 처리")

# 보고서 생성
report = assistant.generate_report()
print("\n" + "="*60)
print("📄 최종 보고서:")
print(report)

## 🎯 실습 과제

### 기본 과제
1. 새로운 도구 3개 추가하기
2. ReAct 에이전트에 메모리 기능 추가
3. 특정 도메인 전문 에이전트 만들기

### 심화 과제
1. LangChain과 통합된 에이전트 구현
2. 웹 스크래핑 도구가 있는 정보 수집 에이전트
3. 코드 생성 및 실행이 가능한 개발자 에이전트

### 프로젝트
1. **개인 비서 에이전트**: 일정 관리, 리마인더, 정보 검색
2. **학습 도우미 에이전트**: 질문 답변, 개념 설명, 퀴즈 생성
3. **프로젝트 관리 에이전트**: 작업 분배, 진행 상황 추적, 보고서 생성

## 📚 추가 학습 자료

- [ReAct 논문](https://arxiv.org/abs/2210.03629)
- [AutoGPT](https://github.com/Significant-Gravitas/AutoGPT)
- [LangChain Agents](https://python.langchain.com/docs/modules/agents/)
- [Microsoft AutoGen](https://github.com/microsoft/autogen)

## 핵심 정리

✅ **AI 에이전트의 핵심 요소**
- 추론(Reasoning): 상황 분석과 판단
- 행동(Acting): 도구 사용과 작업 실행
- 메모리(Memory): 컨텍스트 유지와 학습
- 계획(Planning): 복잡한 작업 분해

✅ **성공적인 에이전트 구축 팁**
1. 명확한 역할과 책임 정의
2. 적절한 도구 제공
3. 효과적인 메모리 관리
4. 지속적인 성능 모니터링
5. 안전장치와 제한 설정

## 마무리

축하합니다! 🎉 

이제 당신은:
- Ollama와 Qwen3를 활용한 로컬 LLM 운영
- LangChain으로 복잡한 워크플로우 구성
- RAG 시스템으로 지식 기반 애플리케이션 구축
- LoRA로 효율적인 모델 커스터마이징
- 자율 에이전트로 복잡한 작업 자동화

를 모두 할 수 있게 되었습니다!

이제 이 지식을 활용하여 자신만의 AI 시스템을 만들어보세요! 🚀